In [3]:
import cv2
import numpy as np
from scipy.stats import skew, kurtosis, entropy

def channel_statistics(image):
    # Split the image into its channels
    b, g, r = cv2.split(image)

    # Filter out black pixels
    non_black_indices = np.where((b > 0) & (g > 0) & (r > 0))

    # Extract non-black values for each channel
    b_non_black = b[non_black_indices]
    g_non_black = g[non_black_indices]
    r_non_black = r[non_black_indices]

    # Calculate mean, variance, skewness, kurtosis, and entropy for each channel
    mean_b, mean_g, mean_r = np.mean(b_non_black), np.mean(g_non_black), np.mean(r_non_black)
    var_b, var_g, var_r = np.var(b_non_black), np.var(g_non_black), np.var(r_non_black)
    skew_b, skew_g, skew_r = skew(b_non_black.flatten()), skew(g_non_black.flatten()), skew(r_non_black.flatten())
    kurt_b, kurt_g, kurt_r = kurtosis(b_non_black.flatten()), kurtosis(g_non_black.flatten()), kurtosis(r_non_black.flatten())
    entropy_b, entropy_g, entropy_r = entropy(b_non_black.flatten()), entropy(g_non_black.flatten()), entropy(r_non_black.flatten())

    # Return the calculated values
    return {
        'mean_b': mean_b, 'mean_g': mean_g, 'mean_r': mean_r,
        'var_b': var_b, 'var_g': var_g, 'var_r': var_r,
        'skew_b': skew_b, 'skew_g': skew_g, 'skew_r': skew_r,
        'kurt_b': kurt_b, 'kurt_g': kurt_g, 'kurt_r': kurt_r,
        'entropy_b': entropy_b, 'entropy_g': entropy_g, 'entropy_r': entropy_r
    }


image = cv2.imread('./data/preprocessed/train/benign/100.jpg')

# Calculate channel statistics
statistics = channel_statistics(image)

# Print the results
for key, value in statistics.items():
    print(f"{key}: {value}")


mean_b: 59.9117053481332
mean_g: 60.331315169862094
mean_r: 104.36293306424487
var_b: 400.08016234574
var_g: 492.6016328818318
var_r: 1162.9183973170805
skew_b: 0.05206825530981384
skew_g: 0.1593895132093942
skew_r: -0.5469023893845275
kurt_b: 1.1575879037070473
kurt_g: 0.7443635165743197
kurt_r: 0.5003534920973269
entropy_b: 8.627357814597048
entropy_g: 8.614848827414448
entropy_r: 8.625865547172634


In [15]:
# Specify the distances and angles for GLCM computation

import cv2

import numpy as np
from featureextraction import calculate_texture_features
%load_ext autoreload
%autoreload 2

image = cv2.imread('./data/preprocessed/train/malignant/10.jpg')
gray_result = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 

result = calculate_texture_features('./data/preprocessed/train/malignant/10.jpg')
result


,Contrast,Energy,Homogeneity,Correlation
0,263.181192,0.757581,0.840391,0.948549


In [31]:



import os
import cv2
import cv2
from skimage.feature import hog
from skimage import exposure
from sklearn.decomposition import PCA

folder_path = "data/preprocessed/test/malignant"
images = []
# Check if the folder exists
if not os.path.exists(folder_path):
    print(f"The folder '{folder_path}' does not exist.")
else:
    # Get a list of all files in the folder
    files = os.listdir(folder_path)

    # Iterate through each file in the folder
    for file_name in files:
        # Construct the full path to the image file
        file_path = os.path.join(folder_path, file_name)

        # Check if the file is a valid image file
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            # Read the image using OpenCV
            image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            resized_image = cv2.resize(image, (112, 122))
            images.append(resized_image)
        else:
            print(f"Skipping non-image file: {file_name}")
  
print(len(images))

image_features = []

for image in images:
    # Compute HOG features
    
    features, _ = hog(image, orientations=8, pixels_per_cell=(8, 8),
                            cells_per_block=(1, 1), visualize=True)

    # Optionally, you can enhance the visualization of HOG features
    image_features.append(features)
    
pca = PCA(n_components=100)
pca_result = pca.fit_transform(image_features)
# make it to dataframe
import pandas as pd
pca_result = pd.DataFrame(pca_result, columns=['HOG_PCA_%i' % i for i in range(100)])
pca_result.head()

300


,HOG_PCA_0,HOG_PCA_1,HOG_PCA_2,HOG_PCA_3,HOG_PCA_4,HOG_PCA_5,HOG_PCA_6,HOG_PCA_7,HOG_PCA_8,HOG_PCA_9,...,HOG_PCA_90,HOG_PCA_91,HOG_PCA_92,HOG_PCA_93,HOG_PCA_94,HOG_PCA_95,HOG_PCA_96,HOG_PCA_97,HOG_PCA_98,HOG_PCA_99
0,-0.484250,3.275221,0.726134,-0.547631,1.389197,-2.078887,-2.176987,2.616156,0.388575,0.005596,...,0.095415,0.369044,0.842613,-1.075634,-0.820823,0.230303,0.032001,-0.020751,0.016192,-0.106166
1,-1.404760,1.070341,-0.352507,-0.079993,1.611410,-0.427155,2.339876,-0.327242,2.110375,-0.864696,...,0.109323,-0.128567,0.039942,-0.202826,0.091228,-0.434165,-0.205828,-0.558451,1.359392,-0.354990
2,4.892988,1.284230,-0.713865,-0.654466,0.499737,-2.905148,1.313615,0.109025,-0.798754,-0.669624,...,0.651620,-0.569649,-0.597299,-0.458099,-0.248295,-0.654915,0.193376,0.360554,0.185571,-0.253594
3,1.646529,-3.270239,-0.078427,-0.188511,1.747291,-0.505466,-0.552271,-1.060779,1.670295,-1.488264,...,-0.307465,-0.144682,-0.843243,-0.220585,-0.271089,-0.100103,-0.134788,0.260975,-0.061967,0.366242
4,-2.461623,-1.668215,-0.330556,-1.283388,1.238189,1.435738,-1.496590,0.806271,0.631519,1.265027,...,0.176803,0.451378,0.150434,0.374808,0.049630,0.306441,0.105744,-0.042925,-0.181290,0.312622


In [35]:
import cv2
import numpy as np
from skimage.feature import greycomatrix
from skimage.measure import shannon_entropy

def calculate_haralick_features(image_path, offset=1):
    # Read the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Calculate the GLCM
    glcm = greycomatrix(image, [offset], [0], symmetric=True, normed=True)

    # Calculate Haralick features
    haralick_features = np.ravel(np.apply_over_axes(np.sum, glcm * np.log(1 + glcm), axes=(0, 1)))

    # Add Shannon entropy as an additional texture feature
    entropy = shannon_entropy(image)
    haralick_features = np.append(haralick_features, entropy)

    return haralick_features



# Example usage
image_path = "data/preprocessed/train/malignant/1001.jpg"
haralick_features = calculate_haralick_features(image_path)
print("Haralick Features:", haralick_features)


Haralick Features: [0.22932874 3.58646012]


c:\Users\drith\anaconda3\lib\site-packages\skimage\feature\__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
